# 09: Running MONAILabel Server for Active Learning with CVAT

In this notebook, we will set up the MONAILabel server for an active learning workflow. This involves connecting the server to CVAT, loading pre-trained models, and setting up active learning loops for iterative fine-tuning and segmentation tasks.

MONAILabel provides the ability to score unlabeled images using methods like **epistemic uncertainty**, select the most uncertain ones, and automatically upload them to CVAT for annotation.

## Key Steps in this Notebook:
1. **Start the MONAILabel server for Active Learning**: We'll connect MONAILabel to CVAT with credentials, load the dataset, and start the segmentation task.
2. **Fine-tune models through active learning**: Set up MONAILabel to identify the most uncertain images and send them to CVAT for annotation.


### Step 1: Launch the MONAILabel Server for Active Learning

We will use Docker to run the MONAILabel server with GPU support and set up active learning. This command mounts your dataset and app folders for MONAILabel to access, and uses port 8080 for CVAT to connect to the MONAILabel server.

Run the following Docker command:

```bash
docker run -it --rm --gpus all --ipc=host --net=host -p 8080:8080 -v /home/exsdatalab/data/endo:/opt/monai/datasets -v /home/exsdatalab/data/apps:/opt/monai/apps projectmonai/monailabel:latest bash
```

Once inside the container, initialize the environment and start the MONAILabel server for active learning.

### **Running the Initialization Script**

Use the helper script `run_monailabelinits.sh` to start the MONAILabel server with all the necessary configurations for active learning:

```bash
./datasets/run_monailabelinits.sh
```

This script does the following:

1. **Environment Variables**:
   ```bash
   export MONAI_LABEL_DATASTORE=cvat
   export MONAI_LABEL_DATASTORE_URL=http://127.0.0.1:8080
   export MONAI_LABEL_DATASTORE_USERNAME=<USERNAME>
   export MONAI_LABEL_DATASTORE_PASSWORD=<PASSWORD>
   ```

2. **Start the MONAILabel Server**:
   ```bash
   monailabel start_server \
     --app apps/endoscopy \
     --studies datasets/images \
     --conf models tooltracking \
     --conf epistemic_enabled true \
     --conf epistemic_top_k 10
   ```

This command will start the MONAILabel server and enable active learning. It will score unlabeled images using epistemic uncertainty and automatically upload the top 10 most uncertain images to CVAT for annotation.

### **Explanation**:
- **MONAI_LABEL_DATASTORE**: Defines that we are using CVAT as the datastore.
- **MONAI_LABEL_DATASTORE_URL**: Specifies the CVAT server URL (running on `http://127.0.0.1:8080`).
- **MONAI_LABEL_DATASTORE_USERNAME and PASSWORD**: Your CVAT login credentials.
- **--app**: Specifies the location of the endoscopic app (e.g., tool tracking).
- **--studies**: Specifies the dataset that MONAILabel will use.
- **--conf models tooltracking**: Loads the pre-trained model for surgical tool segmentation.
- **--conf epistemic_enabled true**: Enables epistemic uncertainty scoring for active learning.
- **--conf epistemic_top_k 10**: Specifies that the top 10 most uncertain images will be selected and uploaded to CVAT for annotation.


### What Happens After the Server Starts

- MONAILabel will begin **scoring** each unlabeled image using **epistemic uncertainty**.
- The server will automatically select the **top 10 most uncertain images** and upload them to CVAT for annotation.
- The **active learning loop** starts: once these images are annotated, MONAILabel will fine-tune the model and continue selecting new uncertain images in subsequent iterations.
- You can view the automatically created projects and tasks in CVAT at `http://127.0.0.1:8080`.

### Next Steps
In the next notebook, we will cover:
1. Accessing projects and tasks in CVAT.
2. Uploading preannotations.
3. Submitting them to MONAILabel to kick-start the active learning workflow.
